In [1]:
import pandas as pd
from matplotlib import colors
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
import numpy as np

# 定义文件路径
file_path = 'vdjdb.txt'  # 将 'your_file.txt' 替换为你的文件路径

# 读取文件内容
with open(file_path, 'r', encoding='utf-8') as file:
    # 读取文件的第一行，获取所有的信息变量名
    header = file.readline().strip().split('\t')
    tcr_data = [dict(zip(header, line.strip().split('\t'))) for line in file]
cdr3_dict = {}
for row in tcr_data:
    complex_id = row['complex.id']
    cdr3 = row['cdr3']
    # Splice together CDR3 with the same complex ID
    if complex_id in cdr3_dict:
        cdr3_dict[complex_id].append(cdr3)
    else:
        cdr3_dict[complex_id] = [cdr3]
# There is a DataFrame containing the TCR sequence
for row in tcr_data:
    complex_id = row['complex.id']
    antigen_epitope = row['antigen.epitope']
    vdjdb_score = row['vdjdb.score']
    # Splice together CDR3 with the same complex ID
    if len(cdr3_dict[complex_id]) == 2:
        cdr3_dict[complex_id].append(antigen_epitope)
        cdr3_dict[complex_id].append(vdjdb_score)
    else:
        continue
cdr3_dict.pop('0')
##Delete unpaired TCRs
df_cdr3 = pd.DataFrame(cdr3_dict)
df_cdr3_trans = df_cdr3.transpose()
names = ['TRA', 'TRB', 'antigen_epitope', 'vdjdb.score']
df_cdr3_trans.columns = names
print(df_cdr3_trans)
##The first step is to read out the paired data

                  TRA                   TRB antigen_epitope vdjdb.score
1       CIVRAPGRADMRF  CASSYLPGQGDHYSNQPQHF        FLKEKGGL           2
2      CAVPSGAGSYQLTF   CASSFEPGQGFYSNQPQHF        FLKEKGGL           2
3         CAVKASGSRLT  CASSYEPGQVSHYSNQPQHF        FLKEKGGL           2
4       CAYRPPGTYKYIF        CASSALASLNEQFF        FLKEKGGL           2
5       CIVRAPGRADMRF  CASSYLPGQGDHYSNQPQHF        FLKEQGGL           2
...               ...                   ...             ...         ...
30590   CMDEGGSNYKLTF         CASSVRSTDTQYF    PQPELPYPQPQL           0
30591     CSLYNNNDMRF         CASSLRYTDTQYF    PQPELPYPQPQL           0
30592   CALSTDSWGKLQF       CASSPGQGGDNEQFF   PQQPFPQPEQPFP           0
30593    CAPQGATNKLIF       CASSLGAGGQETQYF   PQQPFPQPEQPFP           2
30594  CLVGGSGGYNKLIF         CASSSTAQETQYF   PQQPFPQPEQPFP           0

[30594 rows x 4 columns]


In [2]:
df_clean = df_cdr3_trans[df_cdr3_trans['vdjdb.score'] != '0']
#df_clean = df_clean.drop_duplicates()
df_clean = df_clean.reset_index(drop=True)
df_clean['TRA_TRB_Combined'] = df_clean["TRA"] + df_clean["TRB"]
specific_antigen_epitopes = ['PQPELPYPQPQL', 'FLKETGGL','YVLDHLIVV']
df_clean = df_clean[df_clean['antigen_epitope'].isin(specific_antigen_epitopes)]
df_clean = df_clean.reset_index(drop=True)
print("There are {} categories of data in the current dataset".format(np.shape(df_clean['antigen_epitope'].unique())))
print(df_clean)

There are (3,) categories of data in the current dataset
              TRA                   TRB antigen_epitope vdjdb.score  \
0   CIVRAPGRADMRF  CASSYLPGQGDHYSNQPQHF        FLKETGGL           2   
1  CAVPSGAGSYQLTF   CASSFEPGQGFYSNQPQHF        FLKETGGL           2   
2     CAVKASGSRLT  CASSYEPGQVSHYSNQPQHF        FLKETGGL           2   
3     CAYRSAFKLTF       CAWSVPLGRREKLFF       YVLDHLIVV           3   
4  CLVGGDNQGGKLIF        CASSQRQGGNTIYF    PQPELPYPQPQL           2   
5     CIVYNNNDMRF         CASSIRSTDTQYF    PQPELPYPQPQL           2   
6     CIVFNDYKLSF         CASSFRSTDTQYF    PQPELPYPQPQL           2   
7      CIALNARLMF         CASSLRATDTQYF    PQPELPYPQPQL           2   

                    TRA_TRB_Combined  
0  CIVRAPGRADMRFCASSYLPGQGDHYSNQPQHF  
1  CAVPSGAGSYQLTFCASSFEPGQGFYSNQPQHF  
2    CAVKASGSRLTCASSYEPGQVSHYSNQPQHF  
3         CAYRSAFKLTFCAWSVPLGRREKLFF  
4       CLVGGDNQGGKLIFCASSQRQGGNTIYF  
5           CIVYNNNDMRFCASSIRSTDTQYF  
6           CIVFNDYKLSFCASSFRS

In [6]:
##Edit Distance
def ED(str_1, str_2):
    m = len(str_1)
    n = len(str_2)
    # Initializes the dynamic programming matrix with sizes m+1 and n+1 respectively
    Distance = [[0 for _ in range(n + 1)] for _ in range(m + 1)]
    for i in range(n + 1):
        Distance[0][i] = i
    #
    for j in range(m + 1):
        Distance[j][0] = j
    # Initialize the first row and column of the matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            distance_delete = Distance[i - 1][j] + 1
            distance_add = Distance[i][j - 1] + 1
            if str_1[i - 1] == str_2[j - 1]:
                distance_change = Distance[i - 1][j - 1]
            else:
                distance_change = Distance[i - 1][j - 1] + 1
            Distance[i][j] = min(distance_delete, distance_add, distance_change)
    # Count the items from bottom to top
    return Distance[m][n]

In [7]:
##Jaccard Distance
def jaccard_distance(str1, str2):
    set1 = set(str1)
    set2 = set(str2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return 1 - intersection / union

In [8]:
Distance_Matrix_TRA = np.zeros((df_clean.shape[0], df_clean.shape[0]))
Distance_Matrix_TRB = np.zeros((df_clean.shape[0], df_clean.shape[0]))
Distance_Matrix_TRAandTRB = np.zeros((df_clean.shape[0], df_clean.shape[0]))
for i in range(df_clean.shape[0]):
    for j in range(df_clean.shape[0]):
        Distance_Matrix_TRA[i][j] = jaccard_distance(df_clean['TRA'][i], df_clean['TRA'][j])
        Distance_Matrix_TRB[i][j] = jaccard_distance(df_clean['TRB'][i], df_clean['TRB'][j])
        Distance_Matrix_TRAandTRB[i][j] = jaccard_distance(df_clean['TRA_TRB_Combined'][i], df_clean['TRA_TRB_Combined'][j])
    if i % 10 == 0:
        print("Currently, {: 2f} TCR3s have been calculated".format(j))

Currently,  7.000000 TCR3s have been calculated


In [13]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
# Defining functions for clustering distance matrices
def cluster_distance_matrix(distance_matrix,n_cluster=3):
    # Initialize KMeans model
    kmeans = KMeans(n_clusters=3, random_state=42)
    #Fitting distance matrix using KMeans model
    kmeans.fit(distance_matrix)
    #Obtain clustering labels
    labels = kmeans.labels_
    return labels
def cluster_distance_matrix_2(distance_matrix, eps=1.0, min_samples=5):
    #Create DBSCAN model
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    #Fitting distance matrix and clustering
    clusters = dbscan.fit_predict(distance_matrix)
    return clusters.tolist()
def cluster_distance_matrix_3(distance_matrix):
    mean_shift = MeanShift()
    # Fitting and clustering data
    mean_shift.fit(distance_matrix)
    labels = mean_shift.labels_
    return labels
# #Cluster the distance matrix of TRA
TRA_clusters = cluster_distance_matrix(Distance_Matrix_TRA)

# #Cluster the distance matrix of TRB

TRB_clusters=cluster_distance_matrix(Distance_Matrix_TRB)
# #Cluster the distance matrix of TRAandTRB

TRAandTRB_clusters=cluster_distance_matrix(Distance_Matrix_TRAandTRB)

In [14]:
print(TRA_clusters)
print(TRB_clusters)
print(TRAandTRB_clusters)

[1 0 0 0 2 1 2 1]
[2 2 2 0 1 1 1 1]
[2 1 1 1 0 2 0 2]


In [16]:
##Next, calculate the effectiveness of the algorithm
unique_antigen_epitopes = df_clean['antigen_epitope'].unique()
# Create a dictionary corresponding to a category
label_mapping = {antigen_epitope: idx for idx, antigen_epitope in enumerate(unique_antigen_epitopes, start=0)}

# Create a list of real categories
true_labels = [label_mapping[epitope] for epitope in df_clean['antigen_epitope']]

from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
##--------Clustering based on TRAandTRB----------------------------------------
# ARI
ARI_TRAandTRB = adjusted_rand_score(true_labels, TRAandTRB_clusters)
# NMI
NMI_TRAandTRB = normalized_mutual_info_score(true_labels, TRAandTRB_clusters)
#ARI
print("TRAandTRB Adjusted Rand Index (ARI):", ARI_TRAandTRB)
print("TRAandTRB Normalized Mutual Information (NMI):", NMI_TRAandTRB)
##-----------Clustering based on TRA------------------------------------------
# ARI
ARI_TRA = adjusted_rand_score(true_labels, TRA_clusters)
# NMI
NMI_TRA = normalized_mutual_info_score(true_labels, TRA_clusters)
#ARI
print("TRA Adjusted Rand Index (ARI):", ARI_TRA)
print("TRA Normalized Mutual Information (NMI):", NMI_TRA)
##----------------Clustering based on TRB-----------------------------------
# ARI
ARI_TRB = adjusted_rand_score(true_labels, TRB_clusters)
# NMI
NMI_TRB = normalized_mutual_info_score(true_labels, TRB_clusters)
#ARI
print("TRB Adjusted Rand Index (ARI):", ARI_TRB)
print("TRB Normalized Mutual Information (NMI):", NMI_TRB)

TRAandTRB Adjusted Rand Index (ARI): 0.13043478260869565
TRAandTRB Normalized Mutual Information (NMI): 0.4832741472564202
TRA Adjusted Rand Index (ARI): 0.13043478260869565
TRA Normalized Mutual Information (NMI): 0.4832741472564202
TRB Adjusted Rand Index (ARI): 1.0
TRB Normalized Mutual Information (NMI): 1.0
